In [1]:
# Return Period Evalauation
import pandas as pd
import geopandas as gpd
from scipy.signal import find_peaks
import geopandas as gpd
import numpy as np
import pandas as pd
import os
from lmoments3 import distr
os.getcwd()
from scipy.signal import find_peaks
import warnings
import geopandas as gpd
warnings.filterwarnings("ignore", category=RuntimeWarning) 
import re
from shapely.geometry import Point


In [3]:
def extract_peaks_by_year(data, threshold, min_days):
    peaks = []
    data['year'] = data['date'].dt.year

    for year in data['year'].unique():
        yearly_data = data[data['year'] == year].reset_index(drop=True)
        peaks_indices, _ = find_peaks(yearly_data['value'], height=threshold, distance=min_days)
        yearly_peaks = yearly_data.iloc[peaks_indices]

        # Ensure independence between peaks
        valid_peaks = []
        for peak in yearly_peaks.itertuples():
            if not valid_peaks or (peak.date - valid_peaks[-1].date).days >= min_days:
                valid_peaks.append(peak)
        
        if valid_peaks:
            peaks.extend(valid_peaks)
        else:
            continue
    peaks_df = pd.DataFrame(peaks)
    return peaks_df

def GPD_ROI_return_period(peaks,length,shape,thres,scale):
    lambda_u=len(peaks)/length #len(df_grids1.iloc[0][df_grids1.iloc[0]>0])#
    F_u=distr.gpa.cdf(peaks['value']-thres,c=shape,scale=scale)
    return_period=(1/(lambda_u*(1-F_u)))*365.25
    return_period[return_period>1000]=1000
    return return_period



In [16]:
# Read the PPT for 1 Day Precipitation File
df=pd.read_csv(f"Rainfall_SP_1951_2023.csv")
latitudes = df['lat'].values
longitudes = df['lon'].values
df=df.drop(['lat','lon'],axis=1)
df.columns=pd.date_range("1951-01-01",periods=len(df.T),freq="D")
df=df.T[df.T.index.year>=1991].T
geometry = [Point(xy) for xy in zip(latitudes, longitudes)]
df['geometry']=geometry

# Reading the df_threshols from MT-AD Method
df_thres=pd.read_csv("Threshold_MTM_AD_1991_2023.csv")
geometry = [Point(xy) for xy in zip(df_thres['lat'], df_thres['lon'])]
df_thres['geometry']=geometry
# See points inside df_thres
df_thres=df_thres[df_thres['geometry'].isin(df['geometry'])]
# See points insise the data same sequence
df=df[df['geometry'].isin(df_thres['geometry'])]
lat=[];lon=[]
for i in df['geometry']:
    lat.append(i.x)
    lon.append(i.y)

In [17]:

# Evaluating the exceedances
year_df=[];peaks_df=[];exceedances=[]
for s in df_thres['geometry']:
    thres, xi, scale = df_thres.loc[df_thres['geometry'] == s, ['Threshold', 'Shape', 'Scale']].values[0]
    val = df.loc[df['geometry'] == s].drop(columns=['geometry']).T
    data = pd.DataFrame({'date': val.index, 'value': val.values.flatten()}) 
    data['date']=pd.to_datetime(data['date'])
    data = data[(data['date'].dt.month >= 1) & (data['date'].dt.month <= 12)]
    peaks=extract_peaks_by_year(data, thres, min_days=7)[['date','value']]
    exceedances.append(len(peaks)/val.shape[1])
    peaks['geometry']=s
    peaks['return_period']=GPD_ROI_return_period(peaks,length=val.shape[1],shape=xi,thres=thres,scale=scale)       
    peaks_df.append(peaks)

# Evaluate the RP for the specific dates

In [68]:
def GPD_return_period(rate, value, thres, shape, scale):
    # Step 1: GPD CDF for the given value
    if value > thres:
        F_u = distr.gpa.cdf(value - thres, c=shape, scale=scale)
        return_period =(1 / (rate * (1 - F_u)))*365.25
        if return_period>1200:
            return_period=1200
        #return_period[return_period>1000]=1000
    else:
        F_u=0
        return_period = 0* (1 / (rate * (1 - F_u)))*365.25
    return return_period


In [57]:
df_thres['exceedances']=exceedances
df=df.drop('geometry',axis=1)
df_EPE=df.T[(df.T.index>=pd.to_datetime('2023-12-15')) & (df.T.index<=pd.to_datetime('2023-12-20'))].T


KeyError: "['geometry'] not found in axis"

In [69]:
RP_EPE=[]
for j in range(len(df_EPE.T)):
    epe=df_EPE.T.iloc[j].values
    RP=[]
    for i in range(len(df_thres)):
        rate=df_thres.iloc[i]['exceedances']
        value=epe[i]
        shape=df_thres.iloc[i]['Shape']
        threshold=df_thres.iloc[i]['Threshold']
        scale=df_thres.iloc[i]['Scale']
        RP.append(GPD_return_period(rate=rate,value=value,thres=threshold,shape=shape,scale=scale))
    RP_EPE.append(RP)

RP=pd.DataFrame(RP_EPE).T


In [70]:
RP.columns=df_EPE.columns

In [71]:
RP.insert(0,'lat',df_thres['lat'].values)
RP.insert(1,'lon',df_thres['lon'].values)

In [72]:
RP.to_csv("Return_Periods.csv",index=False)

In [61]:
RP

,lat,lon,2023-12-15 00:00:00,2023-12-16 00:00:00,2023-12-17 00:00:00,2023-12-18 00:00:00,2023-12-19 00:00:00,2023-12-20 00:00:00
0,19.00,83.50,0.0,0.0,0.000000,0.000000,0.0,0.0
1,19.00,84.50,0.0,0.0,0.000000,0.000000,0.0,0.0
2,18.75,83.25,0.0,0.0,0.000000,0.000000,0.0,0.0
3,18.75,83.50,0.0,0.0,0.000000,0.000000,0.0,0.0
4,18.75,83.75,0.0,0.0,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...
575,8.50,77.75,0.0,0.0,0.000000,1000.000000,1000.0,0.0
576,8.50,78.00,0.0,0.0,0.000000,1000.000000,1000.0,0.0
577,8.25,77.25,0.0,0.0,2.390132,359.980985,0.0,0.0
578,8.25,77.50,0.0,0.0,6.336896,1000.000000,0.0,0.0
